In [1]:
# 基础
import numpy as np
import pandas as pd
from tqdm import tqdm

# 画图
import matplotlib.pyplot as plt
import seaborn as sns

# 数据
from datasets import load_dataset

# 模型 & pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# 评估
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix



In [2]:
# This cell provides a quick recovery option after kernel restart.
# It does NOT affect the normal execution of the notebook.
# Step 0: 快速恢复（在 Kernel 重启后使用）
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# ---- 1. 恢复 Zero-shot 预测结果 ----
try:
    df_results = pd.read_csv("zero_shot_predictions.csv")
    true_numeric = df_results["true_label"]
    pred_numeric = df_results["pred_label"]
    print("Zero-shot predictions 已成功载入。")
except:
    print("zero_shot_predictions.csv 不存在，请先运行 Zero-shot 推理部分。")

# ---- 2. 恢复模型（从本地文件加载） ----
try:
    local_model_path = r"C:\Users\王茗\Desktop\Final Project\Hugging Face Model"
    tokenizer = AutoTokenizer.from_pretrained(local_model_path, local_files_only=True)
    model = AutoModelForSequenceClassification.from_pretrained(local_model_path, local_files_only=True)
    classifier = pipeline("zero-shot-classification", model=model, tokenizer=tokenizer, device="cpu")
    print("本地 Zero-shot 模型加载成功。")
except:
    print("未找到模型文件夹，请检查 local_model_path。")

print("\n=== Notebook 恢复区执行完毕 ===")


Device set to use cpu


Zero-shot predictions 已成功载入。
本地 Zero-shot 模型加载成功。

=== Notebook 恢复区执行完毕 ===


In [3]:
# Load Yelp Polarity dataset and sample 2000 items
from datasets import load_dataset

ds_test = load_dataset("yelp_polarity", split="test")
ds_sample = ds_test.shuffle(seed=42).select(range(2000))

print("Sample size:", len(ds_sample))


Sample size: 2000


In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# 你的本地模型文件夹路径
local_model_path = r"C:\Users\王茗\Desktop\Final Project\Hugging Face Model"

# 1. 加载 tokenizer（纯本地）
tokenizer = AutoTokenizer.from_pretrained(local_model_path, local_files_only=True)

# 2. 加载 model（纯本地）
model = AutoModelForSequenceClassification.from_pretrained(local_model_path, local_files_only=True)

# 3. 构建 zero-shot pipeline
classifier = pipeline(
    "zero-shot-classification",
    model=model,
    tokenizer=tokenizer,
    device="cpu",
)

print(">>> Model loaded successfully!")
classifier




Device set to use cpu


>>> Model loaded successfully!


In [6]:
#Zero-shot推理
from tqdm import tqdm

pred_labels = []
candidate_labels = ["positive", "negative"]

for item in tqdm(ds_sample):
    result = classifier(
        item["text"],
        candidate_labels=candidate_labels,
        hypothesis_template="This review is {}."
    )
    pred_labels.append(result["labels"][0])




100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [14:35<00:00,  2.28it/s]


In [ ]:
#性能评估
# Step 4: Evaluate zero-shot performance

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Convert string labels to numeric (consistent with yelp dataset)
label_map = {"positive": 1, "negative": 0}
pred_numeric = [label_map[p] for p in pred_labels]
true_numeric = ds_sample["label"]

# Accuracy
print("Zero-shot Accuracy:", accuracy_score(true_numeric, pred_numeric))

# Full Classification Report
print("\nClassification Report:\n")
print(classification_report(true_numeric, pred_numeric))

# Confusion Matrix
cm = confusion_matrix(true_numeric, pred_numeric)
print("\nConfusion Matrix:\n", cm)




In [ ]:
#画混淆矩阵
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, cmap="Blues", fmt="d",
            xticklabels=["Negative","Positive"],
            yticklabels=["Negative","Positive"])
plt.title("Zero-shot Confusion Matrix (DistilBERT MNLI)")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()




In [ ]:
#保存结果
import pandas as pd

df_results = pd.DataFrame({
    "text": ds_sample["text"],
    "true_label": true_numeric,
    "pred_label": pred_numeric
})

df_results.to_csv("zero_shot_predictions.csv", index=False)

df_results.head()


In [ ]:
#Error Analysis
# 错分样本
df_errors = df_results[df_results["true_label"] != df_results["pred_label"]]

# 看 5 条错分例子
df_errors.sample(5, random_state=42)


In [ ]:
#对比图
import matplotlib.pyplot as plt

plt.figure(figsize=(6,4))
plt.bar(["TF-IDF + SVM", "Zero-shot (DistilBERT)"],
        [0.939, 0.789],
        color=["steelblue", "orange"])
plt.ylabel("Accuracy")
plt.title("Accuracy Comparison: TF-IDF vs Zero-shot")
plt.ylim(0,1)
plt.show()

